# Deploy Endpoint

This notebook deploys an endpoint from a ModelPackage.

## Import Important Imports

In [ ]:
import boto3
import sagemaker

## AWS Utils

In [ ]:
def assume_role(
    boto_session: boto3.Session | None,
    role_arn: str,
    session_name: str = "AssumeRoleSession",
    profile_name=None,
) -> boto3.Session:
    """Create a session with assumed role.

    Args:
        boto_session (boto3.Session): The boto3 session to use.
        role_arn (str): The ARN of the role to assume.
        session_name (str): The name of the session.
        profile_name (str): The name of the profile to use.

    Returns:
        boto3.Session: The session with the assumed role.
    """
    if boto_session is None:
        assert profile_name
        # Create a session using the specified profile
        boto_session = boto3.Session(profile_name=profile_name)

    # Create an STS client
    sts_client = boto_session.client("sts")

    # Assume the role
    response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName=session_name)

    # Extract temporary credentials
    credentials = response["Credentials"]

    # Create a new session with the assumed role's credentials
    assumed_session = boto3.Session(
        aws_access_key_id=credentials["AccessKeyId"],
        aws_secret_access_key=credentials["SecretAccessKey"],
        aws_session_token=credentials["SessionToken"],
        region_name="us-east-1",
    )

    return assumed_session


## Constants

In [ ]:
AWS_PROFILE_NAME = "YOU AWS PROFILE HERE"
boto_session = boto3.Session(profile_name=AWS_PROFILE_NAME)

AWS_ACCOUNT_ID = boto_session.client("sts").get_caller_identity()["Account"]
print(f"Using AWS account {AWS_ACCOUNT_ID}")

sm_role = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/AmazonSageMaker-ExecutionRole-Marketplace"
sm_b3_session = assume_role(boto_session, role_arn=sm_role)

sm_session = sagemaker.Session(boto_session=sm_b3_session)

## Deploy Endpoint

In [ ]:
MODEL_PACKAGE_ARN = "YOUR MODEL PACKAGE ARN HERE"

model = sagemaker.ModelPackage(
    role=sm_role,
    model_package_arn=MODEL_PACKAGE_ARN,
    sagemaker_session=sm_session,
)

In [ ]:
ENDPOINT_NAME = "YOUR ENDPOINT NAME HERE"
# 8B on g5.4xlarge ~ 10min
model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.4xlarge",
    endpoint_name=ENDPOINT_NAME,
    sagemaker_session=sm_session,
)
model.endpoint_name